In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path_orders_payment = r'C:\Users\mathe\Projects\clients_seg_database\olist_order_payments_dataset.csv'
file_path_orders = r'C:\Users\mathe\Projects\clients_seg_database\olist_orders_dataset.csv'
file_path_customers = r'C:\Users\mathe\Projects\clients_seg_database\olist_customers_dataset.csv'

In [3]:
df_orders_payment = pd.read_csv(file_path_orders_payment)
df_orders = pd.read_csv(file_path_orders)
df_customers = pd.read_csv(file_path_customers)

In [4]:
df_orders_payment_selected = df_orders_payment[['order_id', 'payment_type', 'payment_installments', 'payment_value']]
df_orders_selected = df_orders[['order_id', 'customer_id', 'order_approved_at']]
df_customers_selected = df_customers[['customer_id', 'customer_unique_id']]

In [5]:
df_merge = pd.merge(
    df_orders_payment_selected,
    df_orders_selected,
    left_on='order_id',
    right_on='order_id',
    how='inner'
)

In [6]:
df_merge_2 = pd.merge(
    df_merge,
    df_customers_selected,
    left_on='customer_id',
    right_on='customer_id',
    how='inner'
)

In [7]:
df_merge_2['order_approved_at'] = pd.to_datetime(df_merge_2['order_approved_at'])
df_merge_2 = df_merge_2[df_merge_2['payment_type'] == 'credit_card']
df_merge_2 = df_merge_2[df_merge_2['order_approved_at'].notna()]

In [8]:
df_recency = (
    df_merge_2.groupby('customer_unique_id', as_index=False)['order_approved_at'].max()
    .sort_values('order_approved_at', ascending=False)
    .reset_index(drop=True)
)

In [9]:
df_recency['recency_score'] = pd.qcut(
    df_recency['order_approved_at'],
    q=5,                              # Divide em 5 grupos (20% cada)
    labels=['1', '2', '3', '4', '5']  # Notas de 1 (menor) a 5 (maior)
)

In [10]:
df_frequency = (
    df_merge_2['customer_unique_id']
    .value_counts()
    .reset_index()
    .rename(columns={
        'count': 'orders_count',
    })
    .sort_values('orders_count', ascending=False)
)

In [11]:
unique_orders = df_frequency['orders_count'].unique()
orders_scoring = pd.qcut(
    unique_orders,
    q=5,
    labels=['1', '2', '3', '4', '5']
)

frequency_score_map = dict(zip(unique_orders, orders_scoring))
df_frequency['frequency_score'] = df_frequency['orders_count'].map(frequency_score_map)
df_frenquency = df_frequency[['customer_unique_id', 'orders_count', 'frequency_score']].copy()

In [12]:
df_payment = df_merge_2[['customer_unique_id', 'order_id', 'payment_installments', 'payment_value']].copy()
df_payment['payment_total'] = df_payment['payment_installments'] * df_payment['payment_value']

In [13]:
df_monetary = (
    df_payment.groupby('customer_unique_id', as_index=False)['payment_total'].sum()
    .sort_values('payment_total', ascending=False)
    .reset_index(drop=True)
)
df_monetary = df_monetary[(df_monetary['payment_total'] > 0.00) & (df_monetary['payment_total'].notna())]

In [14]:
df_monetary['monetary_score'] = pd.qcut(
    df_monetary['payment_total'],
    q=5,
    labels=['1', '2', '3', '4', '5']
)

In [15]:
df_rfm = df_recency.merge(df_frequency, on='customer_unique_id', how='outer') \
             .merge(df_monetary, on='customer_unique_id', how='outer')
df_rfm = df_rfm[['customer_unique_id', 'recency_score', 'frequency_score', 'monetary_score']]

In [16]:
df_rfm = df_rfm.dropna(subset=['recency_score', 'frequency_score', 'monetary_score'])
df_rfm['rfm_score'] = (
    df_rfm['recency_score'].astype(str) + 
    df_rfm['frequency_score'].astype(str) + 
    df_rfm['monetary_score'].astype(str)
)
df_rfm = df_rfm.sort_values('rfm_score', ascending=False)

In [17]:
conditions = [
    (df_rfm['frequency_score'].isin(['4','5'])) & (df_rfm['recency_score'].isin(['4','5'])) & (df_rfm['monetary_score'] == '5'),
    
    (df_rfm['frequency_score'].isin(['4','5'])) & (df_rfm['recency_score'].isin(['4','5'])) & (df_rfm['monetary_score'] == '4'),
    
    (df_rfm['frequency_score'].isin(['3','4'])) & (df_rfm['recency_score'].isin(['3','4'])) & (df_rfm['monetary_score'].isin(['3','4'])),
    
    (df_rfm['frequency_score'].isin(['1','2'])) & (df_rfm['recency_score'].isin(['1','2'])) & (df_rfm['monetary_score'].isin(['1','2']))
]

segments = [
    "Excelência Total",
    "Fidelização Prioritária",
    "Oportunidade de Upsell",
    "Reativação Crítica"
]

df_rfm['segment'] = np.select(conditions, segments, default="Neutros")
df_rfm = df_rfm.reset_index(drop=True)

In [20]:
df_rfm.to_excel("clients_seg.xlsx", index=False)